In [1]:
# ✅ Step 1: Import libraries
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, r2_score
from sklearn.ensemble import RandomForestClassifier, IsolationForest, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import OneClassSVM
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import ipywidgets as widgets
from IPython.display import display
import warnings
warnings.filterwarnings("ignore")

In [2]:
# ✅ Step 2: Load dataset (single path only)
df = pd.read_csv("/content/drive/MyDrive/insider_threat.csv")


In [3]:
# ✅ Step 3: Define target & features
TARGET = "is_insider_threat"
REG_TARGET = "anomaly_score"
X = df.drop(columns=["user_id", TARGET, REG_TARGET])
y = df[TARGET]
y_reg = df[REG_TARGET]

In [4]:
# ✅ Step 4: Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [5]:
# ✅ Step 5: Preprocessing
categorical_features = ["device_type"]
numeric_features = [col for col in X.columns if col not in categorical_features]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ]
)


In [6]:
# ✅ Step 6: Model training (supervised models)
models = {
    "LogisticRegression": Pipeline([
        ("preprocessor", preprocessor),
        ("clf", LogisticRegression(class_weight="balanced", solver="liblinear", random_state=42))
    ]),
    "RandomForest": Pipeline([
        ("preprocessor", preprocessor),
        ("clf", RandomForestClassifier(n_estimators=300, class_weight="balanced", random_state=42))
    ]),
    "XGBoost": Pipeline([
        ("preprocessor", preprocessor),
        ("clf", XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42))
    ])
}

results = {}
for name, pipe in models.items():
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1": f1_score(y_test, y_pred),
    }
    results[name] = {"model": pipe, "metrics": metrics}
    print(f"\n{name} metrics:")
    print(metrics)



LogisticRegression metrics:
{'accuracy': 0.994, 'precision': 0.9397590361445783, 'recall': 0.9873417721518988, 'f1': 0.9629629629629629}

RandomForest metrics:
{'accuracy': 0.997, 'precision': 0.9871794871794872, 'recall': 0.9746835443037974, 'f1': 0.9808917197452229}

XGBoost metrics:
{'accuracy': 0.997, 'precision': 0.975, 'recall': 0.9873417721518988, 'f1': 0.9811320754716981}


In [8]:
# ✅ Step 7 (Fixed): Unsupervised anomaly detection (IsolationForest & OneClassSVM)

# Keep only numeric data (convert categorical manually)
X_encoded = pd.get_dummies(X, columns=["device_type"], drop_first=True)

# Split same as before
X_train_enc, X_test_enc = train_test_split(
    X_encoded, test_size=0.2, stratify=y, random_state=42
)

normal_data = X_train_enc[y_train == 0]

# Scale numeric features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(normal_data)
X_test_scaled = scaler.transform(X_test_enc)

# Isolation Forest
iso = IsolationForest(
    n_estimators=200,
    contamination=y_train.mean(),
    random_state=42
)
iso.fit(X_train_scaled)
iso_pred = (iso.predict(X_test_scaled) == -1).astype(int)

# One-Class SVM
ocsvm = OneClassSVM(kernel="rbf", gamma="scale", nu=y_train.mean())
ocsvm.fit(X_train_scaled)
ocsvm_pred = (ocsvm.predict(X_test_scaled) == -1).astype(int)

# Evaluate metrics
for name, pred in [("IsolationForest", iso_pred), ("OneClassSVM", ocsvm_pred)]:
    metrics = {
        "accuracy": accuracy_score(y_test, pred),
        "precision": precision_score(y_test, pred),
        "recall": recall_score(y_test, pred),
        "f1": f1_score(y_test, pred),
    }
    results[name] = {"model": None, "metrics": metrics}
    print(f"\n{name} metrics:")
    print(metrics)


IsolationForest metrics:
{'accuracy': 0.93, 'precision': 0.5314685314685315, 'recall': 0.9620253164556962, 'f1': 0.6846846846846847}

OneClassSVM metrics:
{'accuracy': 0.916, 'precision': 0.4840764331210191, 'recall': 0.9620253164556962, 'f1': 0.6440677966101694}


In [9]:
# ✅ Step 8: SMOTE + RandomForest if accuracy < 0.9
best_model_name = max(results.keys(), key=lambda k: results[k]["metrics"]["f1"])
best_model = results[best_model_name]["model"]
best_acc = results[best_model_name]["metrics"]["accuracy"]

if best_acc < 0.9:
    print("\nAccuracy < 0.9 detected, applying SMOTE + RandomForest optimization...")
    smote_rf = ImbPipeline([
        ("preprocessor", preprocessor),
        ("smote", SMOTE(random_state=42)),
        ("clf", RandomForestClassifier(n_estimators=400, random_state=42))
    ])
    smote_rf.fit(X_train, y_train)
    y_pred = smote_rf.predict(X_test)
    smote_metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1": f1_score(y_test, y_pred),
    }
    results["SMOTE_RandomForest"] = {"model": smote_rf, "metrics": smote_metrics}
    best_model_name = "SMOTE_RandomForest"
    best_model = smote_rf
    print("\nSMOTE RandomForest metrics:")
    print(smote_metrics)


In [10]:
# ✅ Step 9: Save best model
joblib.dump(best_model, "best_insider_model.joblib")
print(f"\nBest model '{best_model_name}' saved successfully!")


Best model 'XGBoost' saved successfully!


In [11]:
# ✅ Step 10: Train anomaly severity regressor
regressor = Pipeline([
    ("preprocessor", preprocessor),
    ("reg", GradientBoostingRegressor(random_state=42))
])
regressor.fit(X_train, y_reg.loc[y_train.index])
joblib.dump(regressor, "best_anomaly_regressor.joblib")

['best_anomaly_regressor.joblib']

In [12]:
# ✅ Step 11: Test best model
loaded_model = joblib.load("best_insider_model.joblib")
y_pred_final = loaded_model.predict(X_test)
print("\nFinal Model Evaluation Report:")
print(classification_report(y_test, y_pred_final))


Final Model Evaluation Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       921
           1       0.97      0.99      0.98        79

    accuracy                           1.00      1000
   macro avg       0.99      0.99      0.99      1000
weighted avg       1.00      1.00      1.00      1000



In [13]:
# ✅ Step 12: Interactive Prediction UI (Colab widget)
def predict_single_instance():
    input_widgets = {}
    for col in X.columns:
        if col in categorical_features:
            input_widgets[col] = widgets.Dropdown(options=list(df[col].unique()), description=col)
        else:
            input_widgets[col] = widgets.FloatText(value=float(df[col].median()), description=col)

    predict_button = widgets.Button(description="Predict", button_style="success")
    output = widgets.Output()

    def on_predict(b):
        with output:
            output.clear_output()
            single_input = pd.DataFrame({col: [input_widgets[col].value] for col in X.columns})
            model = joblib.load("best_insider_model.joblib")
            reg = joblib.load("best_anomaly_regressor.joblib")

            pred = model.predict(single_input)[0]
            prob = None
            if hasattr(model.named_steps["clf"], "predict_proba"):
                prob = model.predict_proba(single_input)[:, 1][0]

            severity = reg.predict(single_input)[0]

            print("Prediction Result:")
            print(f"Is Insider Threat: {pred}")
            if prob is not None:
                print(f"Probability of Threat: {prob:.3f}")
            print(f"Predicted Anomaly Severity (0-100): {severity:.2f}")

    predict_button.on_click(on_predict)
    display(*input_widgets.values(), predict_button, output)

predict_single_instance()

Dropdown(description='device_type', options=('laptop', 'workstation', 'mobile'), value='laptop')

FloatText(value=794.5472038814969, description='avg_packet_size')

FloatText(value=50.0, description='num_connections')

FloatText(value=10.0, description='unique_ips_accessed')

FloatText(value=0.26333560239340403, description='port_activity_entropy')

...

FloatText(value=0.0, description='phishing_link_clicked')

FloatText(value=173308.18031084846, description='avg_bytes_uploaded')

FloatText(value=0.0, description='geo_location_change')

Button(button_style='success', description='Predict', style=ButtonStyle())

Output()

In [14]:
# ================================
# AI-Driven Insider Threat Detection - Fixed Full Pipeline
# ================================

# Step 1: Imports
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
import ipywidgets as widgets
from IPython.display import display
import warnings
warnings.filterwarnings("ignore")

# Step 2: Load data
df = pd.read_csv("/content/drive/MyDrive/insider_threat.csv")

# Step 3: Define features and targets
TARGET = "is_insider_threat"
REG_TARGET = "anomaly_score"
categorical = ["device_type"]
numeric = [c for c in df.columns if c not in ["user_id", TARGET, REG_TARGET, *categorical]]

X = df[categorical + numeric]
y = df[TARGET]

# Step 4: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42, test_size=0.2)

# Step 5: Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical)
    ]
)

# Step 6: Build model with SMOTE + XGBoost
model = ImbPipeline(steps=[
    ("preprocessor", preprocessor),
    ("smote", SMOTE(random_state=42)),
    ("clf", XGBClassifier(
        use_label_encoder=False,
        eval_metric="logloss",
        scale_pos_weight=(len(y_train) - sum(y_train)) / sum(y_train),  # handle imbalance
        n_estimators=400,
        max_depth=8,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    ))
])

# Step 7: Train model
model.fit(X_train, y_train)

# Step 8: Evaluate
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("Model performance:")
print(f"Accuracy: {acc:.3f}")
print(f"Precision: {precision_score(y_test, y_pred):.3f}")
print(f"Recall: {recall_score(y_test, y_pred):.3f}")
print(f"F1: {f1:.3f}")
print("\nClassification report:\n", classification_report(y_test, y_pred))

# Step 9: Save the model
joblib.dump(model, "best_insider_model.joblib")

# Step 10: Unsupervised anomaly detectors for reference
X_enc = pd.get_dummies(X, columns=categorical, drop_first=True)
X_train_enc, X_test_enc, y_train_enc, y_test_enc = train_test_split(X_enc, y, stratify=y, random_state=42)

scaler = StandardScaler()
normal_data = X_train_enc[y_train_enc == 0]
X_train_scaled = scaler.fit_transform(normal_data)
X_test_scaled = scaler.transform(X_test_enc)

iso = IsolationForest(contamination=y.mean(), random_state=42)
iso.fit(X_train_scaled)
iso_pred = (iso.predict(X_test_scaled) == -1).astype(int)
print("\nIsolationForest F1:", f1_score(y_test_enc, iso_pred))

ocsvm = OneClassSVM(kernel="rbf", gamma="scale", nu=y.mean())
ocsvm.fit(X_train_scaled)
ocsvm_pred = (ocsvm.predict(X_test_scaled) == -1).astype(int)
print("OneClassSVM F1:", f1_score(y_test_enc, ocsvm_pred))

# Step 11: Interactive prediction in Colab
def predict_single_instance():
    input_widgets = {}
    for col in categorical:
        input_widgets[col] = widgets.Dropdown(options=list(df[col].unique()), description=col)
    for col in numeric:
        input_widgets[col] = widgets.FloatText(value=float(df[col].median()), description=col)
    predict_button = widgets.Button(description="Predict", button_style="success")
    output = widgets.Output()

    def on_predict(b):
        with output:
            output.clear_output()
            input_data = pd.DataFrame({col: [input_widgets[col].value] for col in categorical + numeric})
            model = joblib.load("best_insider_model.joblib")
            pred = model.predict(input_data)[0]
            prob = model.predict_proba(input_data)[:, 1][0]
            print("Prediction result:")
            print(f"Insider Threat: {'YES' if pred == 1 else 'NO'}")
            print(f"Threat Probability: {prob:.3f}")

    predict_button.on_click(on_predict)
    display(*input_widgets.values(), predict_button, output)

predict_single_instance()


Model performance:
Accuracy: 0.996
Precision: 0.963
Recall: 0.987
F1: 0.975

Classification report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       921
           1       0.96      0.99      0.97        79

    accuracy                           1.00      1000
   macro avg       0.98      0.99      0.99      1000
weighted avg       1.00      1.00      1.00      1000


IsolationForest F1: 0.7007299270072993
OneClassSVM F1: 0.6808510638297872


Dropdown(description='device_type', options=('laptop', 'workstation', 'mobile'), value='laptop')

FloatText(value=794.5472038814969, description='avg_packet_size')

FloatText(value=50.0, description='num_connections')

FloatText(value=10.0, description='unique_ips_accessed')

...

FloatText(value=0.0, description='phishing_link_clicked')

FloatText(value=173308.18031084846, description='avg_bytes_uploaded')

FloatText(value=0.0, description='geo_location_change')

Button(button_style='success', description='Predict', style=ButtonStyle())

Output()